# The point of this ipynb is to show how one would run a python script (also in this directory) from the command line.  

## The reason for this is because when we submit slurm job arrays, we can provide a single number as an argument. This then becomes "rep" inside the python script, and we need to parse from "rep" what the different parameters should be. This can be tricky to get right.

## First of all, note that bash commands can be given in a jupyter notebook by preceding the command with !  

## Therefore, actual bash commands run on the command line should be the same, but remove the preceding !.

In [1]:
!ls # e.g. show the files here

20221101_random_locations_growthrate.py    medialog_0x7f43a75a9f28
fluxlog_0x7f43a759b588			   RPS_simulations.py
fluxlog_0x7f43a75a9f28			   sys_args_and_rep_to_parms.py
growth_rate_vs_toxins_command_line.ipynb   totalbiomasslog_0x7f43a759b588
growth_rate_vs_toxins_simple_models.ipynb  totalbiomasslog_0x7f43a75a9f28
medialog_0x7f43a759b588			   toxin_and_media_functions.py


In [15]:
# here is the demo file
!tail -n +0 sys_args_and_rep_to_parms.py

import sys
import math

# when a python script is run on the command line like python myscript.py 2,the arguments live in sys.argv. The first arg is the python file and the rest are the following arguments.

args = sys.argv
print("all args:")
print(args)

print("second arg, casted to an int")
rep = int(args[1])
print(rep)

# imagine treatments go here
growth_rates = [0.125, 0.25, 0.5, 1]
space_widths = [0.01, 0.001]

# iterate through these to figure out which to use, and the spatial seed
total_treatments = len(growth_rates) * len(space_widths)
seed = math.floor(rep / total_treatments)
trt_num = rep %  total_treatments
counter = 0
for gr in growth_rates:
    growth_rate = gr
    for sw in space_widths:
        space_width = sw
        if counter == trt_num:
            break
    if counter == trt_num:
        break
print(f"spatial seed = {seed}")
print(f"growth rate = {growth_rate}")
print(f"space width = {space_width}")




In [20]:
# here we should how supplying different arguments gives us different spatial treatment data
for rep in range(8):
    !python sys_args_and_rep_to_parms.py {rep}

system argument = 0
treatment # = 1
spatial seed = 0
growth rate = 0.25
space width = 0.01

system argument = 1
treatment # = 2
spatial seed = 0
growth rate = 0.25
space width = 0.001

system argument = 2
treatment # = 3
spatial seed = 0
growth rate = 0.5
space width = 0.01

system argument = 3
treatment # = 4
spatial seed = 0
growth rate = 0.5
space width = 0.001

system argument = 4
treatment # = 5
spatial seed = 0
growth rate = 1
space width = 0.01

system argument = 5
treatment # = 6
spatial seed = 0
growth rate = 1
space width = 0.001

system argument = 6
treatment # = 1
spatial seed = 1
growth rate = 0.25
space width = 0.01

system argument = 7
treatment # = 2
spatial seed = 1
growth rate = 0.25
space width = 0.001



# To do this on MSI, we'd make a slurm script which called the python script with the supplied argument at the end. Here's an example slurm script which would work for the file above. In this case, I also am loading a few modules, and entering a conda environment. gurobi on MSI is weird and "hijacks" python variables, hence those "unset" lines.

The name of the argument given to the slurm script is $SLURM_ARRAY_TASK_ID, which is then passed to the python script.

In [21]:
!tail -n +0 example_slurm_script.slurm

#!/bin/bash -l
#SBATCH --time=48:00:00
#SBATCH --ntasks=1
#SBATCH --mem=32g
#SBATCH --tmp=10g
#SBATCH --mail-type=NONE

module load conda
module load java/openjdk-13.0.2
module load gurobi/9.0.2
unset PYTHONPATH
unset PYTHONHOME
unset PYTHONSTARTUP

source activate streptomyces

cd RPS

python using_python_scripts_from_command_line_with_arguments.py $SLURM_ARRAY_TASK_ID



# If we put the python script onto MSI in \~/RPS, and the slurm script in home on MSI (\~/), then ran the following code:

sbatch --array=0-120 example_slurm_script.slurm

It would submit jobs to run the python script 120 times, each time with a different argument between 0-120.


0.6250000000000001